#Applied Data Science Capstone Week 3 - 2

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
!wget -q -O 'List_of_postal_codes_of_Canada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# postalcode - geographical coordinates
!wget -q -O 'geopostal.csv' https://cocl.us/Geospatial_data

In [3]:
with open('List_of_postal_codes_of_Canada.html') as html_file:
    soup = BeautifulSoup(html_file,"lxml")

# My assumptions
There is only a table tag in the wiki page.
Text in table's cell is in &lt;td&gt;abc&lt;/td&gt; or &lt;td&gt;&lt;a&gt;abc&lt;/a&gt;&lt;/td&gt;.

In [4]:
table_tag = soup.table
first_row = table_tag.find('tr')
table_header=['PostalCode', 'Borough', 'Neighborhood', 'Latitude','Longitude']


table_values = []

not_assigned = 'Not assigned'


def get_text_in_tag(tag):
    """
    get text in like <td>aaa</td> or <td><a>aaa</a></td> from argument tag.
    """
    if tag.string is not None:
        return tag.string
    else:
        return tag.find('a').string

                             

Read http://cocl.us/Geospatial_data

In [5]:
import csv
postal_to_geographica_coordinates = {}
with open('geopostal.csv') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    
    for index, row in enumerate(csvreader):
        if index > 0 :  postal_to_geographica_coordinates.setdefault(row[0], (row[1],row[2]))
postal_to_geographica_coordinates            

{'M1B': ('43.8066863', '-79.1943534'),
 'M1C': ('43.7845351', '-79.1604971'),
 'M1E': ('43.7635726', '-79.1887115'),
 'M1G': ('43.7709921', '-79.2169174'),
 'M1H': ('43.773136', '-79.2394761'),
 'M1J': ('43.7447342', '-79.2394761'),
 'M1K': ('43.7279292', '-79.2620294'),
 'M1L': ('43.7111117', '-79.2845772'),
 'M1M': ('43.716316', '-79.2394761'),
 'M1N': ('43.692657', '-79.2648481'),
 'M1P': ('43.7574096', '-79.273304'),
 'M1R': ('43.7500715', '-79.2958491'),
 'M1S': ('43.7942003', '-79.2620294'),
 'M1T': ('43.7816375', '-79.3043021'),
 'M1V': ('43.8152522', '-79.2845772'),
 'M1W': ('43.7995252', '-79.3183887'),
 'M1X': ('43.8361247', '-79.2056361'),
 'M2H': ('43.8037622', '-79.3634517'),
 'M2J': ('43.7785175', '-79.3465557'),
 'M2K': ('43.7869473', '-79.385975'),
 'M2L': ('43.7574902', '-79.3747141'),
 'M2M': ('43.789053', '-79.4084928'),
 'M2N': ('43.7701199', '-79.4084928'),
 'M2P': ('43.7527583', '-79.4000493'),
 'M2R': ('43.7827364', '-79.4422593'),
 'M3A': ('43.7532586', '-79.329

Read the 2nd or later rows in table and create dictionary.<br/>
When postalcode as key is in dictionary alaready, add Neighbourhood to array in value of dictionary.

In [6]:
postcode_dict = {} 
for tr in first_row.find_next_siblings():
    row = [ get_text_in_tag(td) if td.string is None else td.string.strip() for td in tr.find_all('td')]

    if row[1] == not_assigned:
        continue
    elif row[2] == not_assigned:
        postcode_dict.setdefault(row[0],  [row[1] ,[] ,postal_to_geographica_coordinates[row[0]]    ])[1].append(row[1])
    else:
        postcode_dict.setdefault(row[0],  [row[1] ,[] ,postal_to_geographica_coordinates[row[0]]   ])[1].append(row[2])
        
data_src = [ [key, value[0], ','.join(value[1]), value[2][0], value[2][1] ] for key,value in postcode_dict.items()]     

Create DataFrame and show it.

In [7]:
df = pd.DataFrame(data_src, columns=table_header)

display(df)    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.3676753
1,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
2,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.6362579,-79.4985091
3,M3A,North York,Parkwoods,43.7532586,-79.3296565
4,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7063972,-79.309937
5,M9N,York,Weston,43.706876,-79.5181884
6,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
7,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
8,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern,43.6627439,-79.321558
9,M1B,Scarborough,"Rouge,Malvern",43.8066863,-79.1943534


In [8]:
df.shape

(103, 5)

end